In [9]:
# Importar librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month
import os
 

In [10]:
spark = SparkSession.builder \
    .appName("SECOP_Ingesta") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/29 01:58:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/29 01:58:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
print(f"Spark Version: {spark.version}")
print(f"Spark Master: {spark.sparkContext.master}")

Spark Version: 3.5.0
Spark Master: local[*]


26/01/29 01:58:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [12]:
import requests
import json

In [13]:
api_url = "https://www.datos.gov.co/resource/jbjy-vk9h.json?$query=SELECT%0A%20%20%60nombre_entidad%60%2C%0A%20%20%60nit_entidad%60%2C%0A%20%20%60departamento%60%2C%0A%20%20%60ciudad%60%2C%0A%20%20%60localizaci_n%60%2C%0A%20%20%60orden%60%2C%0A%20%20%60sector%60%2C%0A%20%20%60rama%60%2C%0A%20%20%60entidad_centralizada%60%2C%0A%20%20%60proceso_de_compra%60%2C%0A%20%20%60id_contrato%60%2C%0A%20%20%60referencia_del_contrato%60%2C%0A%20%20%60estado_contrato%60%2C%0A%20%20%60codigo_de_categoria_principal%60%2C%0A%20%20%60descripcion_del_proceso%60%2C%0A%20%20%60tipo_de_contrato%60%2C%0A%20%20%60modalidad_de_contratacion%60%2C%0A%20%20%60justificacion_modalidad_de%60%2C%0A%20%20%60fecha_de_firma%60%2C%0A%20%20%60fecha_de_inicio_del_contrato%60%2C%0A%20%20%60fecha_de_fin_del_contrato%60%2C%0A%20%20%60condiciones_de_entrega%60%2C%0A%20%20%60tipodocproveedor%60%2C%0A%20%20%60documento_proveedor%60%2C%0A%20%20%60proveedor_adjudicado%60%2C%0A%20%20%60es_grupo%60%2C%0A%20%20%60es_pyme%60%2C%0A%20%20%60habilita_pago_adelantado%60%2C%0A%20%20%60liquidaci_n%60%2C%0A%20%20%60obligaci_n_ambiental%60%2C%0A%20%20%60obligaciones_postconsumo%60%2C%0A%20%20%60reversion%60%2C%0A%20%20%60origen_de_los_recursos%60%2C%0A%20%20%60destino_gasto%60%2C%0A%20%20%60valor_del_contrato%60%2C%0A%20%20%60valor_de_pago_adelantado%60%2C%0A%20%20%60valor_facturado%60%2C%0A%20%20%60valor_pendiente_de_pago%60%2C%0A%20%20%60valor_pagado%60%2C%0A%20%20%60valor_amortizado%60%2C%0A%20%20%60valor_pendiente_de%60%2C%0A%20%20%60valor_pendiente_de_ejecucion%60%2C%0A%20%20%60estado_bpin%60%2C%0A%20%20%60c_digo_bpin%60%2C%0A%20%20%60anno_bpin%60%2C%0A%20%20%60saldo_cdp%60%2C%0A%20%20%60saldo_vigencia%60%2C%0A%20%20%60espostconflicto%60%2C%0A%20%20%60dias_adicionados%60%2C%0A%20%20%60puntos_del_acuerdo%60%2C%0A%20%20%60pilares_del_acuerdo%60%2C%0A%20%20%60urlproceso%60%2C%0A%20%20%60nombre_representante_legal%60%2C%0A%20%20%60nacionalidad_representante_legal%60%2C%0A%20%20%60domicilio_representante_legal%60%2C%0A%20%20%60tipo_de_identificaci_n_representante_legal%60%2C%0A%20%20%60identificaci_n_representante_legal%60%2C%0A%20%20%60g_nero_representante_legal%60%2C%0A%20%20%60presupuesto_general_de_la_nacion_pgn%60%2C%0A%20%20%60sistema_general_de_participaciones%60%2C%0A%20%20%60sistema_general_de_regal_as%60%2C%0A%20%20%60recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_%60%2C%0A%20%20%60recursos_de_credito%60%2C%0A%20%20%60recursos_propios%60%2C%0A%20%20%60ultima_actualizacion%60%2C%0A%20%20%60codigo_entidad%60%2C%0A%20%20%60codigo_proveedor%60%2C%0A%20%20%60fecha_inicio_liquidacion%60%2C%0A%20%20%60fecha_fin_liquidacion%60%2C%0A%20%20%60objeto_del_contrato%60%2C%0A%20%20%60duraci_n_del_contrato%60%2C%0A%20%20%60nombre_del_banco%60%2C%0A%20%20%60tipo_de_cuenta%60%2C%0A%20%20%60n_mero_de_cuenta%60%2C%0A%20%20%60el_contrato_puede_ser_prorrogado%60%2C%0A%20%20%60fecha_de_notificaci_n_de_prorrogaci_n%60%2C%0A%20%20%60nombre_ordenador_del_gasto%60%2C%0A%20%20%60tipo_de_documento_ordenador_del_gasto%60%2C%0A%20%20%60n_mero_de_documento_ordenador_del_gasto%60%2C%0A%20%20%60nombre_supervisor%60%2C%0A%20%20%60tipo_de_documento_supervisor%60%2C%0A%20%20%60n_mero_de_documento_supervisor%60%2C%0A%20%20%60nombre_ordenador_de_pago%60%2C%0A%20%20%60tipo_de_documento_ordenador_de_pago%60%2C%0A%20%20%60n_mero_de_documento_ordenador_de_pago%60%2C%0A%20%20%60documentos_tipo%60%2C%0A%20%20%60descripcion_documentos_tipo%60%0AWHERE%0A%20%20caseless_one_of(%60departamento%60%2C%20%22Distrito%20Capital%20de%20Bogot%C3%A1%22)%0A%20%20AND%20%60fecha_de_firma%60%0A%20%20%20%20%20%20%20%20BETWEEN%20%222025-10-01T19%3A14%3A39%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20%20%20%20AND%20%222025-12-31T23%3A59%3A00%22%20%3A%3A%20floating_timestamp"

In [14]:
response = requests.get(api_url)
data = response.json()

In [17]:
from sodapy import Socrata

client = Socrata("www.datos.gov.co", None)
results = client.get("jbjy-vk9h", query="""
SELECT * 
WHERE 
    departamento="Distrito Capital de Bogotá"
AND
   fecha_de_firma > '2025-11-30T23:59:59'
AND 
    fecha_de_firma < '2026-01-01T00:00:00'
LIMIT   
    10000

    """)

In [18]:
len(results)

10000

In [19]:
# Guardar JSON localmente
json_path = "/opt/spark-data/raw/secop_contratos.json"
os.makedirs(os.path.dirname(json_path), exist_ok=True)

with open(json_path, 'w', encoding='utf-8') as f:
    for record in results:
        f.write(json.dumps(record, ensure_ascii=False) + '\n')

print(f"Datos guardados en: {json_path}")

Datos guardados en: /opt/spark-data/raw/secop_contratos.json


In [21]:
print("Leyendo datos desde JSON...")
df_raw = spark.read.json(json_path)

print(f"Total de registros: {df_raw.count()}")
print(f"Total de columnas: {len(df_raw.columns)}")

Leyendo datos desde JSON...


Total de registros: 10000
Total de columnas: 87


In [22]:
df_raw.head(5)

26/01/29 02:16:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(anno_bpin='2025', c_digo_bpin='2024110010042', ciudad='No Definido', codigo_de_categoria_principal='V1.80111500', codigo_entidad='702271321', codigo_proveedor='703505503', condiciones_de_entrega='A convenir', departamento='Distrito Capital de Bogotá', descripcion_del_proceso='PRESTAR SERVICIOS PROFESIONALES PARA APOYAR EL SEGUIMIENTO Y ANALISIS DE INFORMACION; MEDIANTE HERRAMIENTAS QUE PERMITAN EL CONTROL Y CONSOLIDACION DE LAS ACTIVIDADES Y PROCESOS RELACIONADOS CON LOS \nSERVICIOS Y ESTRATEGIAS DE LA PRIMERA INFANCIA; INFANCIA Y LA ADOLESCENCIA DE LA SDIS EN EL MARCO DE', descripcion_documentos_tipo='No definido', destino_gasto='Inversión', dias_adicionados='0', documento_proveedor='1013637109', documentos_tipo='No', domicilio_representante_legal='No Definido', duraci_n_del_contrato='5 Mes(es)', el_contrato_puede_ser_prorrogado='No', entidad_centralizada='Centralizada', es_grupo='No', es_pyme='No', espostconflicto='No', estado_bpin='Válido', estado_contrato='Aprobado', fecha_de_

In [23]:
df_raw.printSchema()

root
 |-- anno_bpin: string (nullable = true)
 |-- c_digo_bpin: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- codigo_entidad: string (nullable = true)
 |-- codigo_proveedor: string (nullable = true)
 |-- condiciones_de_entrega: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- descripcion_documentos_tipo: string (nullable = true)
 |-- destino_gasto: string (nullable = true)
 |-- dias_adicionados: string (nullable = true)
 |-- documento_proveedor: string (nullable = true)
 |-- documentos_tipo: string (nullable = true)
 |-- domicilio_representante_legal: string (nullable = true)
 |-- duraci_n_del_contrato: string (nullable = true)
 |-- el_contrato_puede_ser_prorrogado: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- es_grupo: string (nullable = true)
 |-- es_pyme: string (nullable = true)
 |-- e

In [24]:
from pyspark.sql.functions import col, regexp_replace, when
from pyspark.sql.types import DecimalType, LongType

numeric_decimal_cols = [
    "valor_amortizado",
    "valor_de_pago_adelantado",
    "valor_del_contrato",
    "valor_facturado",
    "valor_pagado",
    "valor_pendiente_de",
    "valor_pendiente_de_ejecucion",
    "valor_pendiente_de_pago",
    "saldo_cdp",
    "saldo_vigencia",
    "presupuesto_general_de_la_nacion_pgn",
    "sistema_general_de_participaciones",
    "sistema_general_de_regal_as",
    "recursos_de_credito",
    "recursos_propios"
]

numeric_long_cols = [
    "dias_adicionados",
    "codigo_entidad",
    "nit_entidad"
]

df_silver = df_raw

# Decimal (montos)
for c in numeric_decimal_cols:
    df_silver = df_silver.withColumn(
        c,
        when(col(c).isNull() | (col(c) == ""), None)
        .otherwise(
            regexp_replace(col(c), ",", "").cast(DecimalType(18, 2))
        )
    )

# Long (IDs / días)
for c in numeric_long_cols:
    df_silver = df_silver.withColumn(
        c,
        when(col(c).isNull() | (col(c) == ""), None)
        .otherwise(col(c).cast(LongType()))
    )


In [25]:
from pyspark.sql.functions import to_timestamp

timestamp_cols = [
    "fecha_de_fin_del_contrato",
    "fecha_de_firma",
    "fecha_de_inicio_del_contrato",
    "fecha_de_notificaci_n_de_prorrogaci_n",
    "ultima_actualizacion",
    "fecha_fin_liquidacion",
    "fecha_inicio_liquidacion"
]

for c in timestamp_cols:
    df_silver = df_silver.withColumn(
        c,
        when(col(c).isNull() | (col(c) == ""), None)
        .otherwise(to_timestamp(col(c)))
    )


In [26]:
df_silver.printSchema()

root
 |-- anno_bpin: string (nullable = true)
 |-- c_digo_bpin: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- codigo_entidad: long (nullable = true)
 |-- codigo_proveedor: string (nullable = true)
 |-- condiciones_de_entrega: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- descripcion_documentos_tipo: string (nullable = true)
 |-- destino_gasto: string (nullable = true)
 |-- dias_adicionados: long (nullable = true)
 |-- documento_proveedor: string (nullable = true)
 |-- documentos_tipo: string (nullable = true)
 |-- domicilio_representante_legal: string (nullable = true)
 |-- duraci_n_del_contrato: string (nullable = true)
 |-- el_contrato_puede_ser_prorrogado: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- es_grupo: string (nullable = true)
 |-- es_pyme: string (nullable = true)
 |-- espos